# WiDS University Datathon — Starter Notebook (Joins on Sample, Smaller, Dataset)

This notebook runs **external data joins** using **small sample heads** of the WatchDuty datasets.

- NOAA NCEI (GHCND) — daily weather by nearest station
- OpenStreetMap (Overpass) — infrastructure near perimeter centroids

> To run on full data later, download from Kaggle and replace file paths and parse columns to find relevant data for your project.


In [9]:
# If running in Colab, install deps
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install pandas shapely requests

import os, time, math
from pathlib import Path
import pandas as pd
import requests

from shapely import wkt
print('Imports ready. pandas=', pd.__version__)


Imports ready. pandas= 2.2.2


## Load small sample datasets
These are 5-row heads stored under `data/examples/`.

In [10]:
DATA = Path("/content")

PERIMS = DATA / "perimeter_sample.csv"
EVENTS = DATA / "geo_events_sample.csv"

# Load the uploaded CSVs
perims = pd.read_csv(PERIMS)
events = pd.read_csv(EVENTS)

# Helpers

def parse_wkt_strip_srid(geom_str):
    if pd.isna(geom_str):
        return None
    text = str(geom_str)
    if text.startswith('SRID='):
        text = text.split(';', 1)[-1]
    try:
        return wkt.loads(text)
    except Exception:
        return None

# Centroids from perimeter polygons
perims['geometry'] = perims['geom'].map(parse_wkt_strip_srid)
perims['lat'] = perims['geometry'].map(lambda g: g.centroid.y if g is not None else None)
perims['lon'] = perims['geometry'].map(lambda g: g.centroid.x if g is not None else None)

# Normalize event dates
date_col = 'date_modified' if 'date_modified' in events.columns else 'date_created'
events['_event_dt'] = pd.to_datetime(events[date_col], errors='coerce', utc=True).dt.normalize()

perims.head(), events.head()

(   id            date_created           date_modified     source_date_current  \
 0   1  2023-07-01 12:00:00+00  2023-07-01 13:00:00+00  2023-07-01 12:30:00+00   
 1   2  2023-07-10 09:00:00+00  2023-07-10 10:30:00+00  2023-07-10 09:15:00+00   
 2   3  2023-07-15 16:00:00+00  2023-07-15 16:45:00+00  2023-07-15 16:10:00+00   
 3   4  2023-08-03 08:00:00+00  2023-08-03 08:40:00+00  2023-08-03 08:20:00+00   
 4   5  2023-08-20 14:30:00+00  2023-08-20 15:05:00+00  2023-08-20 14:45:00+00   
 
       source_incident_name                                               geom  \
 0           Sample Fire LA  SRID=4326;POLYGON((-118.30 34.05,-118.20 34.05...   
 1           Sample Fire SF  SRID=4326;POLYGON((-122.45 37.75,-122.35 37.75...   
 2      Sample Fire Redding  SRID=4326;POLYGON((-122.45 40.55,-122.35 40.55...   
 3  Sample Fire Bakersfield  SRID=4326;POLYGON((-119.10 35.35,-119.00 35.35...   
 4    Sample Fire San Diego  SRID=4326;POLYGON((-117.20 32.70,-117.10 32.70...   
 
            

## NOAA NCEI (GHCND) — Daily Weather Join
Requires a free token from https://www.ncdc.noaa.gov/cdo-web/token

Once you have the token, replace the text: YOUR_TOKEN_HERE
in the single quotes with your actual token

In [12]:
os.environ['NOAA_TOKEN'] = 'hnTIwfKBRgKJfTPqYXBDrZwaZPdyrXHc'

In [13]:
# --- NOAA daily join (robust, chunked, works with new minimal CSVs) ---

import os, time, math, requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Required columns: id, date_created, date_modified, geo_event_type, name, lat, lng

NCEI_BASE = "https://www.ncei.noaa.gov/cdo-web/api/v2"

# ---------- Resilient HTTP session ----------
def _make_session(retries=5, backoff=1.0):
    """
    Create a requests Session with retries & exponential backoff.
    Retries on: 429, 500, 502, 503, 504, plus connection/read errors.
    """
    retry = Retry(
        total=retries,
        connect=retries,
        read=retries,
        backoff_factor=backoff,                  # exponential (1s, 2s, 3s, …)
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    s = requests.Session()
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://", HTTPAdapter(max_retries=retry))
    return s

def ncei_request(path, params, token, timeout=(10, 120), session=None):
    """
    Single NCEI call with robust retry & larger timeouts.
    timeout=(connect, read)
    """
    session = session or _make_session()
    url = f"{NCEI_BASE}/{path}"
    headers = {"token": token}

    # short manual retries for ReadTimeout/ConnectionError
    manual_retries = 2
    for i in range(manual_retries + 1):
        try:
            r = session.get(url, headers=headers, params=params, timeout=timeout)
            r.raise_for_status()
            return r.json()
        except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as e:
            if i < manual_retries:
                time.sleep(2 * (i + 1))
                continue
            raise
        except requests.HTTPError:
            # Non-2xx after urllib3 retries—surface the body
            raise RuntimeError(f"NCEI HTTP {r.status_code}: {r.text[:500]}")

# ---------- Small geo helpers ----------
def bbox_from_point(lat, lon, km=50):
    dlat = km / 111.0
    dlon = km / (111.0 * max(0.1, math.cos(math.radians(lat))))
    return (lon - dlon, lat - dlat, lon + dlon, lat + dlat)

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    from math import radians, sin, cos, asin, sqrt
    p1, p2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlmb = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(p1)*cos(p2)*sin(dlmb/2)**2
    return 2 * R * asin(sqrt(a))

# ---------- Station search ----------
def find_nearest_station(lat, lon, start, end, token, datasetid="GHCND", km=50, session=None):
    """
    Query NCEI 'stations' around (lat, lon) within bbox, then pick closest.
    """
    session = session or _make_session()
    minlon, minlat, maxlon, maxlat = bbox_from_point(lat, lon, km)
    js = ncei_request(
        "stations",
        {
            "datasetid": datasetid,
            "extent": f"{minlat},{minlon},{maxlat},{maxlon}",
            "startdate": start,
            "enddate": end,
            "limit": 1000,
        },
        token,
        session=session,
    )
    results = js.get("results", []) if isinstance(js, dict) else []
    if not results:
        return None

    best = min(
        results,
        key=lambda s: haversine_km(lat, lon, s.get("latitude"), s.get("longitude")),
    )
    best = dict(best)
    best["dist_km"] = haversine_km(lat, lon, best["latitude"], best["longitude"])
    return best

# ---------- Chunked daily fetch ----------
def fetch_daily_chunked(stationid, start, end, token,
                         datatypes=("TMAX","TMIN","PRCP","WSF2"),
                         chunk_days=30, limit=1000, session=None):
    """
    Fetch DAILY observations in chunks to avoid slow/large requests.
    Returns a wide DataFrame: date + columns per datatype.
    """
    session = session or _make_session()
    start_dt = pd.to_datetime(start).normalize()
    end_dt   = pd.to_datetime(end).normalize()

    frames = []
    current = start_dt
    while current <= end_dt:
        chunk_end = min(current + pd.Timedelta(days=chunk_days-1), end_dt)
        params = {
            "datasetid": "GHCND",
            "stationid": stationid,
            "startdate": current.strftime("%Y-%m-%d"),
            "enddate":   chunk_end.strftime("%Y-%m-%d"),
            "units": "standard",
            "limit": limit,
            "datatypeid": list(datatypes),   # requests repeats this key
        }
        try:
            js = ncei_request("data", params, token, timeout=(10, 120), session=session)
            res = js.get("results", []) if isinstance(js, dict) else []
            if res:
                frames.append(pd.DataFrame(res))
        except Exception as e:
            print(f"[WARN] NOAA chunk {current.date()}..{chunk_end.date()} failed: {e}")
        current = chunk_end + pd.Timedelta(days=1)

    if not frames:
        return pd.DataFrame(columns=["date"])

    df = pd.concat(frames, ignore_index=True).drop_duplicates()
    # Normalize date, pivot wide
    df["date"] = pd.to_datetime(df["date"], utc=True, errors="coerce").dt.tz_convert(None).dt.normalize()
    df = df.pivot_table(index="date", columns="datatype", values="value", aggfunc="first").reset_index()
    df.columns.name = None
    return df

# ---------- Demo using the *events* file (has lat/lng) ----------
# Load events if not already present
if "events" not in globals():
    events = pd.read_csv("/mnt/data/geo_events_sample_final.csv")

# Parse event timestamps
events["_event_dt"] = pd.to_datetime(
    events["date_modified"].fillna(events["date_created"]),
    utc=True, errors="coerce"
).dt.tz_convert(None)

# Derive the window strictly from the data; if missing, STOP.
evt_min = events["_event_dt"].min()
evt_max = events["_event_dt"].max()
if pd.isna(evt_min) or pd.isna(evt_max):
    raise ValueError(
        "No valid event timestamps found. "
        "Please fix the data OR set explicit START_DATE/END_DATE in the notebook."
    )

start = evt_min.strftime("%Y-%m-%d")
end   = evt_max.strftime("%Y-%m-%d")

# Pick a representative point (median lat/lng across events)
events_nonnull = events.dropna(subset=["lat","lng"]).copy()
events_nonnull["lat"] = pd.to_numeric(events_nonnull["lat"], errors="coerce")
events_nonnull["lng"] = pd.to_numeric(events_nonnull["lng"], errors="coerce")
mid_lat = float(events_nonnull["lat"].median())
mid_lng = float(events_nonnull["lng"].median())

# NOAA token (required)
token = os.environ.get("NOAA_TOKEN")
if not token:
    raise SystemExit("Please set NOAA_TOKEN first (free token: https://www.ncdc.noaa.gov/cdo-web/token).")

# Find nearest station and fetch chunked daily weather
station = find_nearest_station(mid_lat, mid_lng, start, end, token, km=50)
if station is None:
    raise SystemExit("No nearby GHCND station found; try increasing km or adjusting dates.")

daily = fetch_daily_chunked(
    stationid=station["id"],
    start=start,
    end=end,
    token=token,
    datatypes=("TMAX","TMIN","PRCP","WSF2"),  # adjust as needed
    chunk_days=30,
    limit=1000
)
if daily.empty:
    raise SystemExit("No NOAA daily data returned for this station/date window.")

# Prepare keys and join by date
daily["noaa_date"] = pd.to_datetime(daily["date"], errors="coerce").dt.date
events["_event_date"] = events["_event_dt"].dt.date

joined = events.merge(daily, left_on="_event_date", right_on="noaa_date", how="left")

# Small preview
display(pd.DataFrame([station]))
display(daily.head())
display(joined.head())
print(f"Joined shape: {joined.shape}")

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,dist_km
0,143.3,1999-01-01,2025-09-05,35.4186,"BAKERSFIELD 5 NW, CA US",0.9441,GHCND:USC00040444,METERS,-119.0508,4.292737


,date,PRCP,TMAX,TMIN,noaa_date
0,2023-07-01,0.0,104.0,77.0,2023-07-01
1,2023-07-02,0.0,105.0,75.0,2023-07-02
2,2023-07-03,0.0,104.0,75.0,2023-07-03
3,2023-07-04,0.0,106.0,71.0,2023-07-04
4,2023-07-05,0.0,100.0,69.0,2023-07-05


,id,date_created,date_modified,geo_event_type,name,lat,lng,_event_dt,_event_date,date,PRCP,TMAX,TMIN,noaa_date
0,1,2023-07-01 12:00:00.000000,2023-07-01 13:00:00.000000,fire,Sample Fire LA,34.07,-118.25,2023-07-01 13:00:00,2023-07-01,2023-07-01,0.00,104.0,77.0,2023-07-01
1,2,2023-07-10 09:00:00.000000,2023-07-10 10:30:00.000000,fire,Sample Fire SF,37.77,-122.40,2023-07-10 10:30:00,2023-07-10,2023-07-10,0.00,90.0,62.0,2023-07-10
2,3,2023-07-15 16:00:00.000000,2023-07-15 16:45:00.000000,fire,Sample Fire Redding,40.58,-122.39,2023-07-15 16:45:00,2023-07-15,2023-07-15,0.00,98.0,74.0,2023-07-15
3,4,2023-08-03 08:00:00.000000,2023-08-03 08:40:00.000000,fire,Sample Fire Bakersfield,35.38,-119.05,2023-08-03 08:40:00,2023-08-03,2023-08-03,0.00,102.0,67.0,2023-08-03
4,5,2023-08-20 14:30:00.000000,2023-08-20 15:05:00.000000,fire,Sample Fire San Diego,32.72,-117.15,2023-08-20 15:05:00,2023-08-20,2023-08-20,0.02,101.0,70.0,2023-08-20


Joined shape: (5, 14)


## OpenStreetMap (Overpass) — Infrastructure Nearby
No key required. Queries hospitals, schools, police, and fire stations near the first few centroids.

In [14]:
import requests

OVERPASS = 'https://overpass-api.de/api/interpreter'
TAGS = [('amenity','hospital'),('amenity','school'),('amenity','police'),('amenity','fire_station')]

def overpass_query(lat, lon, radius_m=5000):
    parts = []
    for k,v in TAGS:
        parts.append(f'node["{k}"="{v}"](around:{int(radius_m)},{lat},{lon});')
        parts.append(f'way["{k}"="{v}"](around:{int(radius_m)},{lat},{lon});')
        parts.append(f'relation["{k}"="{v}"](around:{int(radius_m)},{lat},{lon});')
    body = f"[out:json][timeout:25];({''.join(parts)});out center;"
    r = requests.post(OVERPASS, data=body, timeout=60)
    r.raise_for_status()
    js = r.json()
    rows = []
    for el in js.get('elements', []):
        tags = el.get('tags', {})
        lat_ = el.get('lat') or (el.get('center') or {}).get('lat')
        lon_ = el.get('lon') or (el.get('center') or {}).get('lon')
        if lat_ is None or lon_ is None: continue
        rows.append({'osm_id': f"{el.get('type')}:{el.get('id')}", 'name': tags.get('name'), 'amenity': tags.get('amenity'), 'lat': lat_, 'lon': lon_})
    return pd.DataFrame(rows)

subset = perims.dropna(subset=['lat','lon']).head(3).copy()
frames = []
for _, r in subset.iterrows():
    df = overpass_query(float(r['lat']), float(r['lon']), radius_m=5000)
    df['perim_id'] = r.get('id')
    frames.append(df)
    time.sleep(1.2)
if frames:
    osm = pd.concat(frames, ignore_index=True)
    counts = osm.groupby(['perim_id','amenity']).size().reset_index(name='count')
    display(osm.head())
    display(counts.head())
else:
    print('No OSM features returned. Try a larger radius.')


,osm_id,name,amenity,lat,lon,perim_id
0,node:358780625,Sal Castro Middle School,school,34.062789,-118.262574,1
1,node:358780633,Bernedo Middle School,school,34.049734,-118.294242,1
2,node:358781672,Castelar Elementary School,school,34.064243,-118.239990,1
3,node:358782407,Commonwealth Elementary School,school,34.069455,-118.285909,1
4,node:358782779,Cutler Academy,school,34.069178,-118.289798,1


,perim_id,amenity,count
0,1,fire_station,16
1,1,hospital,16
2,1,police,9
3,1,school,140
4,2,fire_station,28
